In [ ]:
# импорт библиотек

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

!pip3 install transformers
!pip3 install torch
!pip install catboost

import re # Регулярные выражения
import torch # PyTorch
import transformers # для инициализации токенизатора, объект класса BertTokenizer() и прочие трансформеры
from tqdm import notebook # индикатор прогресса
from sklearn.linear_model import LogisticRegression # логистическая регрессия
from sklearn.tree import DecisionTreeClassifier # дерево-классификатор
from sklearn.model_selection import cross_val_score # cv
from sklearn.model_selection import train_test_split # разбика выборок
from sklearn.model_selection import RandomizedSearchCV # подбор гиперпараметров и cv
from sklearn.metrics import f1_score # F1

import nltk # NLP инструменты
from nltk.corpus import stopwords # списки стоп-слов
nltk.download('stopwords') # загрузили списко стоп-слов
stop_words = set(stopwords.words('english')) # пометили, что на английском стоп-слова
from sklearn.feature_extraction.text import TfidfVectorizer # импорт TF-IDF
count_tf_idf = TfidfVectorizer(stop_words=list(stop_words)) # счетчик TF-IDF
from sklearn.feature_extraction.text import CountVectorizer # импорт bow
count_vect = CountVectorizer(stop_words=list(stop_words)) # счетчик bow

from transformers import AutoTokenizer, AutoModelForMaskedLM # ваниант вместо Bert
# model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

from catboost import CatBoostClassifier # классификатор
from sklearn.utils.class_weight import compute_class_weight # чтобы посчитать балланс классов для Catbo

nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer # лемматайзер
# Create WordNetLemmatizer object
# wnl = WordNetLemmatizer()

from nltk import word_tokenize # токенайзер
nltk.download('punkt') # пунктуация для токенайзер 
nltk.download('averaged_perceptron_tagger') # POSS tagg из коробки

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.5/922.5 KB 34.3 MB/s eta 0:00:00
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.0.9
    Uninstalling spacy-3.0.9:
      Successfully uninstalled spacy-3.0.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 3.5.1 which is incompatible.


In [ ]:
# установили обновление spacy c Git
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 63.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached spacy-3.0.9-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
  Using cached thinc-8.0.17-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (668 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.9
    Uninstalling thinc-8.1.9:
      Successfully uninstalled thinc-8.1.9
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.1
    Uninstalling spacy-3.5.1:
      Successfully uninstalled spacy-3.5.1


In [ ]:
# установили flair
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.3/374.3 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 37.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 KB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 KB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import flair

In [ ]:
# задали модель flair для сентимента
model = flair.models.TextClassifier.load('en-sentiment')

2023-03-26 15:41:33,497 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmp9pdfwsxr


100%|██████████| 253M/253M [00:12<00:00, 21.8MB/s]

2023-03-26 15:41:46,078 copying /tmp/tmp9pdfwsxr to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2023-03-26 15:41:47,333 removing temp file /tmp/tmp9pdfwsxr


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# функция для разрезания длинного текста на куски с пересечением 

def get_overlapped_chunks(long_text, chunksize, overlapsize):  
    return [ long_text[a:a+chunksize] for a in range(0,len(long_text), chunksize-overlapsize)]

UnicodeDecodeError: 'utf-8' codec can't decode byte in position: invalid continuation byte

encoding='latin-1'

In [ ]:
# пример выведения длинного текста их фала формата .txt
long_text = ''.join(open('/Users/user/My Python Stuff/example_news.txt', encoding='latin-1').readlines())

In [ ]:
print(long_text)


Russia-Ukraine War: Is it all fair in love and war? What about brands leveraging conflicts and wars to push their wares?
In 2016, three days before the 15th anniversary of the 9/11 attack, Texas-based mattress brand Miracle issued an apology to the victims and families. Why? What did a mattress brand have to do with the terror attack?
Miracle was aggressively promoting its ÔTwin Tower saleÕ. As if that werenÕt bad enough, it also released an ad that spoofed the bringing down of the towers. Naturally, these tasteless marketing tactics brought on a massive backlash against the brand. So much so that the company had to shut shop soon after.
In the same year and month, when the Indian Army conducted a surgical strike against terrorist camps across the Line of Control in Pakistani-administered Kashmir - a reaction to a deadly terrorist attack that killed 19 Indian soldiers, several brands leveraged IndiaÕs surgical strike. But there was one that stood out.
Gurgaon-based burger brand Burger 

In [ ]:
# загрузка базы с текстами на колаб
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Индия ИТОГ-2022-11-01T15-19-10.691089.csv to Индия ИТОГ-2022-11-01T15-19-10.691089.csv
User uploaded file "Индия ИТОГ-2022-11-01T15-19-10.691089.csv" with length 2789630 bytes


In [ ]:
# чтение scv c текстами
corpus = pd.read_csv('Индия ИТОГ-2022-11-01T15-19-10.691089.csv')

In [ ]:
# посмотрим
display(corpus.head())

,id,parent_id,title,publication_author,publication_date,external_url,internal_url,text,views_count
0,ОК-5415486,NaN,'Key international actor': India wins global p...,Last Updated Sep ; Source ANI,2022-09-26 00:00:00,https://zeenews.india.com/world/key-internatio...,NaN,'Key international actor': India wins global p...,NaN
1,ОК-5415252,ОК-5415486,9405__1094963-india-un.jpg,NaN,NaN,https://english.cdn.zeenews.com/sites/default/...,/minio/crawlers/images%2Ffull%2Fb69e%2F9405__1...,NaN,NaN
2,ОК-5415997,NaN,'VERY WELL PUT': German envoy HAILS PM Narendr...,Ritesh K Srivastava; Last Updated Sep ; Source...,2022-09-26 00:00:00,https://zeenews.india.com/india/very-well-put-...,NaN,'VERY WELL PUT': German envoy HAILS PM Narendr...,NaN
3,ОК-5415761,ОК-5415997,e8c6__1095157-modi-putin-lead.jpg,NaN,NaN,https://english.cdn.zeenews.com/sites/default/...,/minio/crawlers/images%2Ffull%2F7ff6%2Fe8c6__1...,NaN,NaN
4,ОК-5417656,NaN,'Possibility of sabotage could not be ruled ou...,Zee Media Bureau; Last Updated Sep ; Source Re...,2022-09-27 00:00:00,https://zeenews.india.com/international-busine...,NaN,'Possibility of sabotage could not be ruled ou...,NaN


In [ ]:
# пропуски
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1257 entries, 0 to 1256
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1257 non-null   object 
 1   parent_id           632 non-null    object 
 2   title               1257 non-null   object 
 3   publication_author  601 non-null    object 
 4   publication_date    625 non-null    object 
 5   external_url        1257 non-null   object 
 6   internal_url        632 non-null    object 
 7   text                533 non-null    object 
 8   views_count         0 non-null      float64
dtypes: float64(1), object(8)
memory usage: 88.5+ KB


In [ ]:
# удалим лишние столбцы
corpus = corpus.drop(['parent_id', 'publication_author', 'external_url', 'internal_url', 'views_count'],axis=1)

In [ ]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1257 entries, 0 to 1256
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                1257 non-null   object
 1   title             1257 non-null   object
 2   publication_date  625 non-null    object
 3   text              533 non-null    object
dtypes: object(4)
memory usage: 39.4+ KB


In [ ]:
# удалим все строки, в которых есть пропуски в столбцах 'text' и 'publication_date'
corpus = corpus.dropna(subset=['text', 'publication_date'])

In [ ]:
# посмотрим на чистый корпус
display(corpus.head(10))

,id,title,publication_date,text
0,ОК-5415486,'Key international actor': India wins global p...,2022-09-26 00:00:00,'Key international actor': India wins global p...
2,ОК-5415997,'VERY WELL PUT': German envoy HAILS PM Narendr...,2022-09-26 00:00:00,'VERY WELL PUT': German envoy HAILS PM Narendr...
4,ОК-5417656,'Possibility of sabotage could not be ruled ou...,2022-09-27 00:00:00,'Possibility of sabotage could not be ruled ou...
12,ОК-5202439,दुनिया,2022-09-14 11:53:00,World News\nUSA News: पेंटागन के प्रवक्ता ब्रि...
25,ОК-5094892,Immense potential for energy cooperation betwe...,2022-09-07 13:32:00,There is immense potential for cooperation bet...
34,ОК-4606215,India Has Never Been Defensive About Its Stand...,2022-08-17 22:48:59,EAM Jaishankar said that oil prices are unreas...
38,ОК-4859210,India to incentivize rupee-settled exports to ...,2022-08-24 00:00:00,"2 min read.Updated: 24 Aug 2022, 06:23 PM ISTN..."
49,ОК-5237436,"India, Russia To Ensure Uninterrupted Defence ...",2022-09-04 19:09:11,Russian Ambassador Denis Alipov's comments cam...
51,ОК-5237858,"Sheikh Hasina In India: United In Trade, Resil...",2022-09-05 23:10:07,India played an important role in Bangladesh’s...
53,ОК-5219005,India's August Sunflower Oil Imports Up 89.6%,2022-09-13 17:56:18,"India, the world's leading vegetable oil buyer..."


In [ ]:
# создадим функцию, которая пройдет по всем текстам и оценит их тональность
def get_sentiment(text):
    # токанизация подаваемого текста
    sentence = flair.data.Sentence(text)
    # предсказание сентимента токенизированного текста
    model.predict(sentence)
    # из результата берем лемматизированный текст и количественную оценку сентимента 
    sentiment = sentence.labels[0]
    return sentiment

In [ ]:
# с контролем времени выполнения запустим функцию выше 
# результат запишем в новый столбец 'sentiment' 
notebook.tqdm.pandas()
corpus['sentiment'] = corpus['text'].progress_apply(get_sentiment)

  0%|          | 0/425 [00:00<?, ?it/s]

In [ ]:
# посмотрим как отработал
display(corpus.head())

,id,title,publication_date,text,sentiment
0,ОК-5415486,'Key international actor': India wins global p...,2022-09-26 00:00:00,'Key international actor': India wins global p...,"Sentence[865]: ""'Key international actor': Ind..."
2,ОК-5415997,'VERY WELL PUT': German envoy HAILS PM Narendr...,2022-09-26 00:00:00,'VERY WELL PUT': German envoy HAILS PM Narendr...,"Sentence[719]: ""'VERY WELL PUT': German envoy ..."
4,ОК-5417656,'Possibility of sabotage could not be ruled ou...,2022-09-27 00:00:00,'Possibility of sabotage could not be ruled ou...,"Sentence[975]: ""'Possibility of sabotage could..."
12,ОК-5202439,दुनिया,2022-09-14 11:53:00,World News\nUSA News: पेंटागन के प्रवक्ता ब्रि...,"Sentence[2752]: ""World News USA News: पेंटागन ..."
25,ОК-5094892,Immense potential for energy cooperation betwe...,2022-09-07 13:32:00,There is immense potential for cooperation bet...,"Sentence[415]: ""There is immense potential for..."


In [ ]:
# установка spacy
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spacy

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
# задали модель nlp (spacy для английского) для выделения NER
nlp = spacy.load("en_core_web_sm")

In [ ]:
# создадим функцию, которая вернет список NER нужного типа 
def get_countr(text):
    # пропустим тексты чере модель nlp (spacy для английского) для выделения NER  
    doc = nlp(text)
    # создадим пустой список, в который будем добавлять NER нужного типа 
    countr_list = []
    # цикл пройдет по всем NER текста и добавит в пустой список только NER с тэгом 'GPE' 
    for entity in doc.ents:
        if entity.label_ == 'GPE':
            countr_list.append(entity.text)
    # если NER с тэгом 'GPE' была распознана более одного раза в список она будет добавлена соответствующее количество раз
    # используем метод set() чтобы удалить дубли и затем в список
    countr_list = list(set(countr_list))
    return countr_list

In [ ]:
# со счетчиком времени запустим функцию выше
# результат работы функции запишем в новый столбец 'GPE_NER'
notebook.tqdm.pandas()
corpus['GPE_NER'] = corpus['text'].progress_apply(get_countr)

  0%|          | 0/425 [00:00<?, ?it/s]

In [ ]:
# посмотрим как отработала
display(corpus.head())

,id,title,publication_date,text,sentiment,GPE_NER
0,ОК-5415486,'Key international actor': India wins global p...,2022-09-26 00:00:00,'Key international actor': India wins global p...,"Sentence[865]: ""'Key international actor': Ind...","[New York, US, UNGA, Brazil, Portugal, Samarka..."
2,ОК-5415997,'VERY WELL PUT': German envoy HAILS PM Narendr...,2022-09-26 00:00:00,'VERY WELL PUT': German envoy HAILS PM Narendr...,"Sentence[719]: ""'VERY WELL PUT': German envoy ...","[New Delhi, US, China, Washington, the United ..."
4,ОК-5417656,'Possibility of sabotage could not be ruled ou...,2022-09-27 00:00:00,'Possibility of sabotage could not be ruled ou...,"Sentence[975]: ""'Possibility of sabotage could...","[Moscow, Norway, Denmark, Warsaw, Ukraine, Pol..."
12,ОК-5202439,दुनिया,2022-09-14 11:53:00,World News\nUSA News: पेंटागन के प्रवक्ता ब्रि...,"Sentence[2752]: ""World News USA News: पेंटागन ...","[अपनी-, Denmark, Florida, बरसी, चुनौती होगी, अ..."
25,ОК-5094892,Immense potential for energy cooperation betwe...,2022-09-07 13:32:00,There is immense potential for cooperation bet...,"Sentence[415]: ""There is immense potential for...","[Ukraine, Russia, India]"


In [ ]:


corpus['GPE_NER'] = corpus['GPE_NER'].apply(lambda x: ast.literal_eval(str(x)))

In [ ]:
import ast

In [ ]:
display(corpus.head())

,id,title,publication_date,text,sentiment,GPE_NER
0,ОК-5415486,'Key international actor': India wins global p...,2022-09-26 00:00:00,'Key international actor': India wins global p...,"Sentence[865]: ""'Key international actor': Ind...","[New York, US, UNGA, Brazil, Portugal, Samarka..."
2,ОК-5415997,'VERY WELL PUT': German envoy HAILS PM Narendr...,2022-09-26 00:00:00,'VERY WELL PUT': German envoy HAILS PM Narendr...,"Sentence[719]: ""'VERY WELL PUT': German envoy ...","[New Delhi, US, China, Washington, the United ..."
4,ОК-5417656,'Possibility of sabotage could not be ruled ou...,2022-09-27 00:00:00,'Possibility of sabotage could not be ruled ou...,"Sentence[975]: ""'Possibility of sabotage could...","[Moscow, Norway, Denmark, Warsaw, Ukraine, Pol..."
12,ОК-5202439,दुनिया,2022-09-14 11:53:00,World News\nUSA News: पेंटागन के प्रवक्ता ब्रि...,"Sentence[2752]: ""World News USA News: पेंटागन ...","[अपनी-, Denmark, Florida, बरसी, चुनौती होगी, अ..."
25,ОК-5094892,Immense potential for energy cooperation betwe...,2022-09-07 13:32:00,There is immense potential for cooperation bet...,"Sentence[415]: ""There is immense potential for...","[Ukraine, Russia, India]"


In [ ]:
# создадим словарь с NER и оценками тональности текстов, в которых они встречаются
sentiment = {}

# цикл пройдет по значениям и строкам датасета и извлечет NER и оценку тональности для словаря
for i, row in corpus.iterrows():
    # извлечение NER и оценки тональности 
    direction = row['sentiment'].value
    score = row['sentiment'].score
    # вложенный цикл пройдет по всем NER в столбце 'GPE_NER' 
    for ner in row['GPE_NER']:
        # проверит добавлена ли уже NER из строки в словарь sentiment
        if ner not in sentiment.keys():
            # если не добавлена, то добавить ее в словарь со значениями положительной и отрицательной оценки тональности (flair дает две) 
            sentiment[ner] = {'POSITIVE': [], 'NEGATIVE': []}
        # добавление в словарь уникальных (это обеспечивает вложенный цикл) NER c тэгом GRE и сентементами в словарь sentiment
        sentiment[ner][direction].append(score)

In [ ]:
# выведем оценки тональности для NER 'Russia' 
display(sentiment['Russia'])

{'POSITIVE': [0.961162805557251,
  0.6064042448997498,
  0.5092902183532715,
  0.5171924829483032,
  0.9790711402893066,
  0.785873532295227,
  0.7778164148330688,
  0.6282986998558044,
  0.958103597164154,
  0.975140392780304,
  0.7365450263023376,
  0.8767128586769104,
  0.9638592600822449,
  0.9781791567802429,
  0.9418255090713501,
  0.9537302851676941,
  0.882615864276886,
  0.9986550807952881,
  0.5621554255485535,
  0.588372528553009,
  0.7761282324790955,
  0.9989196062088013,
  0.9875850677490234,
  0.9875850677490234,
  0.9875850677490234,
  0.9952968955039978,
  0.962478756904602,
  0.9521661996841431,
  0.7671682834625244,
  0.8068727850914001,
  0.794300377368927,
  0.8738666772842407,
  0.8481778502464294,
  0.8924823999404907,
  0.9124024510383606,
  0.5181029438972473,
  0.9858554005622864,
  0.5205222964286804,
  0.8869274854660034,
  0.5490552186965942,
  0.6988360285758972,
  0.9873524904251099,
  0.6322401762008667,
  0.8917326331138611,
  0.7829445004463196,
  0.97

In [ ]:
# создадим пустой список, в который положим среднюю оценку тональности для каждой NER из словаря выше
avg_sentiment = []

# цикл пройдет по всем NER, т.е. ключам словаря sentiment.keys()
for ner in sentiment.keys():
    # частота = кол-во положительных оценко + кол-во отрицательных оценко
    freq = len(sentiment[ner]['POSITIVE']) + len(sentiment[ner]['NEGATIVE'])
    for direction in ['POSITIVE', 'NEGATIVE']:
        # назначим переменную
        score = sentiment[ner][direction]
        # если количество оценок для данной NER = 0
        if len(score) == 0:
            sentiment[ner][direction] = 0.0 
        else:
            # иначе просуммировать оценки
            sentiment[ner][direction] = sum(score)
    # итоговая оценка = положительная - отрицательная
    total = sentiment[ner]['POSITIVE'] - sentiment[ner]['NEGATIVE']
    # и разделить на частоту
    avg = total/freq
    # добавим эти данные в созданный выше пустой список 
    avg_sentiment.append({
        'entity': ner,
        'positive': sentiment[ner]['POSITIVE'],
        'negative': sentiment[ner]['NEGATIVE'],
        'frequency': freq,
        'score': avg
    })

In [ ]:
# создадим датафрей, в который положим список выше
sentiment_df = pd.DataFrame(avg_sentiment)
# посмотрим
sentiment_df.head()

,entity,positive,negative,frequency,score
0,New York,7.503024,22.964430,32,-0.483169
1,US,23.983865,169.762384,204,-0.714601
2,UNGA,0.854659,0.000000,1,0.854659
3,Brazil,6.429636,15.493509,24,-0.377661
4,Portugal,3.817414,0.998425,5,0.563798


In [ ]:
# оставим в списке только те NER, которые встречались более 3 раз
sentiment_df = sentiment_df[sentiment_df['frequency'] > 3]
sentiment_df

,entity,positive,negative,frequency,score
0,New York,7.503024,22.964430,32,-0.483169
1,US,23.983865,169.762384,204,-0.714601
3,Brazil,6.429636,15.493509,24,-0.377661
4,Portugal,3.817414,0.998425,5,0.563798
5,Samarkand,9.389888,13.675361,26,-0.164826
...,...,...,...,...,...
431,Bengal,2.962755,1.767532,5,0.239045
487,Xinjiang,0.817256,3.886173,5,-0.613783
488,Hangzhou,0.000000,3.996116,4,-0.999029
490,Zhejiang province,0.000000,3.996116,4,-0.999029


In [ ]:
from google.colab import files
sentiment_df.to_csv('GRE_sentiments_from_India_lenta.csv') 
files.download('GRE_sentiments_from_India_lenta.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# отранжировали значения по положительным оценкам
sentiment_df.sort_values('positive', ascending=False).head(20)

,entity,positive,negative,frequency,score
8,India,58.339548,268.710211,348,-0.604513
16,Russia,52.046083,250.437585,321,-0.618042
11,China,32.003414,160.666707,206,-0.624579
14,Ukraine,29.649668,206.633226,249,-0.710777
1,US,23.983865,169.762384,204,-0.714601
18,Moscow,14.976230,89.459531,112,-0.665029
93,Uzbekistan,11.177459,19.645550,35,-0.241945
9,France,10.280339,12.451110,24,-0.090449
5,Samarkand,9.389888,13.675361,26,-0.164826
35,Pakistan,8.820997,53.513941,68,-0.657249
